# Initialization

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,ShuffleSplit
import seaborn as sns
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm

In [2]:
file_path = 'Normal_MC.xlsx'
df = pd.read_excel(file_path)
df = df.iloc[:,1:]

##### Making Features and Target Matrices

The dataset contaain both structural descripotrs and physicsal descripotors, here we are just going to consider the Physical Descripotrs

In [3]:
L = [281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344]

XX = df.iloc[:,L] #Has shape 40000*60 (4000* 10 configurations)
YY = df.iloc[:,3] #Has shape 40000*1  (4000* 10 configurations)

# Information Imbalance: 

In [6]:
def compute_information_imbalance(distances_A, distances_B, k=1):
    N = distances_A.shape[0]
    
    ranks_A = np.argsort(distances_A, axis=1)
    ranks_B = np.argsort(distances_B, axis=1)
    

    imbalance_A_to_B = compute_imbalance(ranks_A, ranks_B, N, 1)
    imbalance_B_to_A = compute_imbalance(ranks_B, ranks_A, N, 1)
    
    return imbalance_A_to_B, imbalance_B_to_A

def compute_imbalance(ranks_from, ranks_to, N = 4000, l = 1): # l here corresponding to k in information imbalance paper, and we are calculating imbakllance for k=1

    nn_indices = ranks_from[:, 1:l+1] 
    

    nn_ranks_in_to = np.array([
        [np.where(ranks_to[i] == nn_indices[i, j])[0][0] for j in range(l)]
        for i in range(N)
    ])
    
    avg_rank = np.mean(nn_ranks_in_to) / N
    imbalance = 2 * avg_rank
    
    return imbalance

def compute_symmetric_imbalance(dist_all, selected_descriptors):
    combined_data = np.hstack([d1[:,j].reshape(-1, 1) for j in selected_descriptors])
    dist_combined = squareform(pdist(combined_data))
    imbalance_all_combined = compute_information_imbalance(dist_all, dist_combined)
    symmetric_imbalance = (imbalance_all_combined[0] + imbalance_all_combined[1]) / np.sqrt(2)
    return symmetric_imbalance , imbalance_all_combined[0] , imbalance_all_combined[1]

## Supervised One

comparing descripotrs space with propensity space

In [ ]:
descriptors_cnf = []
imbalance_jp_cnf = []
imbalance_pj_cnf = []


for cnf in range(1,11):
    
    X =XX.iloc[4000*(cnf-1):4000*cnf]
    y = YY.iloc[4000*(cnf-1):4000*cnf]
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    y = np.array(y)  
    


    dist_p = squareform(pdist(y.reshape(-1,1)))
    best_descriptors = []
    imbalance_jp = []
    imbalance_pj = []
    best_j_p = float('inf')

    
    #Calculating 1st Descriptor
    for j in range(0,60):

        dist_j = squareform(pdist(X[:, j].reshape(-1, 1)))
        imbalance_J_to_P, imbalance_P_to_J = compute_information_imbalance(dist_j , dist_p)

        if imbalance_J_to_P < best_j_p : 
            best_j_p = imbalance_J_to_P
            same_p_j = imbalance_P_to_J
            best_descriptors = [j]


    imbalance_jp.append(best_j_p)
    imbalance_pj.append(same_p_j)        
    #print(f"best J_P = {best_j_p}")
    #print(f"Same P_J = {same_p_j}")
    #print(f"best_descriptors = {best_descriptors}")


    #Calculating Remaining Descriptors
    for _ in range(14):
        best_j_p = float('inf') 
        current_best_descriptor = None 

        for j in range(0,60):
            if j in best_descriptors:
                continue

            candidate_descriptors = best_descriptors + [j] 
            combined_data = np.hstack([X[:,j].reshape(-1, 1) for j in candidate_descriptors])

            dist_new = squareform(pdist(combined_data))      
            imbalance_J_to_P, imbalance_P_to_J = compute_information_imbalance(dist_new, dist_p)

            if imbalance_J_to_P < best_j_p: 
                best_j_p = imbalance_J_to_P
                same_p_j = imbalance_P_to_J 
                current_best_descriptor = j

        if current_best_descriptor is not None:
            best_descriptors.append(current_best_descriptor)
            best_imbalance =  best_j_p
            imbalance_jp.append(best_j_p)
            imbalance_pj.append(same_p_j)
            print(f"Updated best symmetric_imbalance: {best_imbalance}")
            print(f"same p_j = {same_p_j}")
            print(f"Selected descriptors: {best_descriptors}")
    
    
    # Adding Random Number descriptor
    random_descriptor = np.random.rand(X.shape[0], 1)
    selected_descriptors = np.hstack([X[:, best_descriptors], random_descriptor]) 
    dist_combined = squareform(pdist(selected_descriptors))
    imbalance_J_to_P, imbalance_P_to_J = compute_information_imbalance(dist_combined, dist_p)
    best_descriptors.append(1000)
    # index 1000 is random number vector
    
    imbalance_jp.append(imbalance_J_to_P)
    imbalance_pj.append(imbalance_P_to_J)
    
    print(f"Imbalance with random descriptor - J to P: {imbalance_J_to_P}")
    print(f"Imbalance with random descriptor - P to J: {imbalance_P_to_J}")
    print(f"Final descriptors (including random): {best_descriptors}")
    
    descriptors_cnf.append(best_descriptors)
    imbalance_jp_cnf.append(imbalance_jp)
    imbalance_pj_cnf.append(imbalance_pj)

Saving Results

In [ ]:
arr_jp = np.array(imbalance_jp_cnf)
arr_pj = np.array(imbalance_pj_cnf)
arr_cnf = np.array(descriptors_cnf)

Arr_imb_sup = np.vstack([arr_jp,arr_pj,arr_cnf])

#Arr_imb_sup contain first 10 columns as arr_jp, next 10 as arr_pj and last 10 as arr_cnf 
np.save("Innformation_imbalance_supervised", Arr_imb_sup)

In [ ]:
#To load array, use this
Imbalance_supervised = np.load("Innformation_imbalance_supervised.npy")

## Unsupervised one

Comparing the space of Descripotrs with a smaller subset of fewer descripotrs among them

In [ ]:
best_descrip_all = []
best_imbalance_all = []
best_pj_all = []
best_jp_all = []

for cnf in range(1,11):
    
    df_new =XX.iloc[4000*(cnf-1):4000*cnf]
    scaler = StandardScaler()
    d1 = scaler.fit_transform(df_new)
    
    
    # Example usage
    np.random.seed(42)
    best_descriptors = []
    sym_imbalance = []
    best_pj = []
    best_jp = []
    
    dist_all = squareform(pdist(d1))
    best_symmetric_imbalance = float('inf')
    
    


    for j in range(0,60):
        dist_j = squareform(pdist(d1[:, j].reshape(-1, 1)))
        imbalance_all_j = compute_information_imbalance(dist_all, dist_j)
        symmetric_imbalance = (imbalance_all_j[0] + imbalance_all_j[1]) / np.sqrt(2)

        if symmetric_imbalance < best_symmetric_imbalance:
            best_symmetric_imbalance = symmetric_imbalance
            best_descriptors = [j]
            sym_imbalance = [best_symmetric_imbalance]
            best_pj = [imbalance_all_j[0]]
            best_jp = [imbalance_all_j[1]]
    

    for _ in range(1, 15):
        current_best_symmetric_imbalance = float('inf')
        current_best_descriptor = None

        for j in range(0,60):
            if j in best_descriptors:
                continue

            candidate_descriptors = best_descriptors + [j]
            symmetric_imbalance , pj , jp = compute_symmetric_imbalance(dist_all, candidate_descriptors)

            if symmetric_imbalance < current_best_symmetric_imbalance:
                current_best_symmetric_imbalance = symmetric_imbalance
                current_best_descriptor = j
                current_best_pj = pj
                current_best_jp = jp

        if current_best_descriptor is not None:
            best_descriptors.append(current_best_descriptor)
            best_pj.append(current_best_pj)
            best_jp.append(current_best_jp)
            
            sym_imbalance.append(current_best_symmetric_imbalance)
            best_symmetric_imbalance = current_best_symmetric_imbalance
           
    print(f"Final best symmetric_imbalance: {best_symmetric_imbalance}")
    print(f"Final selected descriptors: {best_descriptors}")
    #print(best_pj)
    #print(best_jp)
    
    best_pj_all.append(best_pj)
    best_jp_all.append(best_jp)
    best_descrip_all.append(best_descriptors)
    best_imbalance_all.append(sym_imbalance)

Saving Data

In [ ]:
desc_arr = np.array(best_descrip_all)
imbalance_arr = np.array(best_imbalance_all)
pj_array = np.array(best_pj_all)
jp_array = np.array(best_jp_all)

imbalance_unsup = np.vstack([desc_arr,imbalance_arr ,pj_array,jp_array])
np.save("Innformation_imbalance_unsupervised", imbalance_unsup)

In [ ]:
imbalance_unsup = np.load("Innformation_imbalance_unsupervised.npy")

## Information_imbalance_each_descriptor_vs_propensity 

In [ ]:
descriptors_cnf = []
imbalance_jp_cnf = []
imbalance_pj_cnf = []


for cnf in range(1,11):
    
    X =XX.iloc[4000*(cnf-1):4000*cnf]
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    Y = np.array(YY.iloc[4000*(cnf-1):4000*cnf])
    
    dist_p = squareform(pdist(Y.reshape(-1,1)))
    best_descriptors = []
    imbalance_jp = []
    imbalance_pj = []
    best_j_p = float('inf')

    for j in range(0,60):

        dist_j = squareform(pdist(X[:, j].reshape(-1, 1)))
        imbalance_J_to_P, imbalance_P_to_J = compute_information_imbalance(dist_j , dist_p)
        imbalance_jp.append(imbalance_J_to_P)
        imbalance_pj.append(imbalance_P_to_J)        
        
    imbalance_jp_cnf.append(imbalance_jp)
    imbalance_pj_cnf.append(imbalance_pj)
    print(cnf)

Saving Data

In [ ]:
arr_jp = np.array(imbalance_jp_cnf)
arr_pj = np.array(imbalance_pj_cnf)

Arr_des_prop  = np.vstack([arr_jp,arr_pj]) 
np.save("Innformation_imbalance_each_descripotr_vs_propensity", Arr_des_prop)

In [ ]:
Arr_des_prop = np.load("Innformation_imbalance_each_descripotr_vs_propensity.npy")

# Regression via various methods

## Lasso Selection

In [ ]:
import numpy as np
from sklearn.linear_model import Lasso, Ridge
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

def find_alpha_for_lasso(X_train, y_train, num_features, tol=1e-4, max_iter=1000):
    alpha = 1.0
    step = alpha / 2.0
    selected_features = 0
    
    for _ in range(max_iter):
        lasso = Lasso(alpha=alpha, tol=tol)
        lasso.fit(X_train, y_train)
        selected_features = np.sum(lasso.coef_ != 0)
        
        if selected_features == num_features:
            break
        elif selected_features < num_features:
            alpha -= step
        else:
            alpha += step
        step /= 2.0
    
    return alpha, lasso

def lasso_ridge_debiasing(X_train, y_train, X_test, y_test, num_features, alpha_ridge=0):
    # Normalize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    
    alpha_lasso, lasso = find_alpha_for_lasso(X_train, y_train, num_features)
    
    if np.sum(lasso.coef_ != 0) != num_features:
        print(f"Could not find alpha to select exactly {num_features} features. Selected {np.sum(lasso.coef_ != 0)} instead.")
    
    selected_features = np.where(lasso.coef_ != 0)[0]

    if len(selected_features) == 0:
        print("No features selected by Lasso. Try reducing num_features.")
        return None

    # Ridge Regression on selected features
    X_train_ridge = X_train[:, selected_features]
    X_test_ridge = X_test[:, selected_features]
    
    ridge = Ridge(alpha=alpha_ridge)
    ridge.fit(X_train_ridge, y_train)
    ridge_coef = ridge.coef_

    y_pred = ridge.predict(X_test_ridge)
    pearson_corr, _ = pearsonr(y_test, y_pred)
    
    return pearson_corr


results_combined = []

for cnf in range(1,11):
    
    print(cnf)
    X_test = XX.iloc[4000*(cnf-1):4000*cnf, :]
    y_test = YY.iloc[4000*(cnf-1):4000*cnf]

    X_train = XX.drop(XX.index[4000*(cnf-1):4000*cnf])
    y_train = YY.drop(YY.index[4000*(cnf-1):4000*cnf])
    
    # Apply LassoRidge Debiasing algorithm for different numbers of selected features
    num_features_list = np.arange(1,16)
    results = []


    for num_features in num_features_list:
        result = lasso_ridge_debiasing(X_train, y_train, X_test, y_test, num_features, alpha_ridge=0)
        if result:
            results.append(result)

    
    results_combined.append(results)

In [ ]:
Debias_result = np.array(results_combined)
np.save("Debias_result", Debias_result)

## Supervised Info. Imbalance

In [ ]:
pearson_coff = []

for cnf in range(1, 11):
    print(f"Processing configuration {cnf}")
    pear = []
    arr = np.array(Imbalance_supervised[19+cnf,:15])
    XXX = XX.iloc[:,arr]
    
    X_test = XXX.iloc[4000*(cnf-1):4000*cnf, :]
    y_test = YY.iloc[4000*(cnf-1):4000*cnf]
    
    X_train = XXX.drop(XXX.index[4000*(cnf-1):4000*cnf])
    y_train = YY.drop(YY.index[4000*(cnf-1):4000*cnf])
    
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_train)
    X_te = scaler.transform(X_test)
    
    for m in range(1,16):
        
        
        X_tr_sel = X_tr[:,:m]#.reshape(-1,1)
        X_te_sel = X_te[:,:m]#.reshape(-1,1)
        #print(X_tr_sel)

        ridge = Ridge(alpha = 0)
        ridge.fit(X_tr_sel,y_train)

        y_pr = ridge.predict(X_te_sel)
        pearson_corr_p, p_value = pearsonr( y_pr,y_test)
        pear.append(abs(pearson_corr_p))
        
    pearson_coff.append(pear)
    print("cnf:" , cnf)
    

In [ ]:
Info_imb_sup = np.array(pearson_coff) 
np.save("Info_imb_sup", Info_imb_sup) 

## Unsupervised Info. Imbalance

In [ ]:
pearson_coff = []

for cnf in range(1, 11):
    print(f"Processing configuration {cnf}")
    pear = []
    arr = np.array(imbalance_unsup[cnf-1,:])
    XXX = XX.iloc[:,arr]
    
    # Split the data into training (4000) and testing (36000) sets
    X_test = XXX.iloc[4000*(cnf-1):4000*cnf, :]
    y_test = YY.iloc[4000*(cnf-1):4000*cnf]
    
    X_train = XXX.drop(XXX.index[4000*(cnf-1):4000*cnf])
    y_train = YY.drop(YY.index[4000*(cnf-1):4000*cnf])
    
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_train)
    X_te = scaler.transform(X_test)
    
    for m in range(1,16):
        
        
        X_tr_sel = X_tr[:,:m]#.reshape(-1,1)
        X_te_sel = X_te[:,:m]#.reshape(-1,1)
        #print(X_tr_sel)

        ridge = Ridge(alpha = 0)
        ridge.fit(X_tr_sel,y_train)

        y_pr = ridge.predict(X_te_sel)
        pearson_corr_p, p_value = pearsonr( y_pr,y_test)
        pear.append(abs(pearson_corr_p))
        
    pearson_coff.append(pear)
    print("cnf:" , cnf)
    

In [ ]:
Info_imb_unsup = np.array(pearson_coff)
np.save("Info_imb_unsup", Info_imb_unsup)

## Random Selecton

In [ ]:
pearson_coff = []

for cnf in range(1, 11):
    print(f"Processing configuration {cnf}")
    pear = []
    
    # Repeating the experiment 1000 times
    for _ in range(1000):
        
        arr = np.random.choice(60, size=60, replace=False)
        XXX = XX.iloc[:, arr]
        
        X_test = XXX.iloc[4000*(cnf-1):4000*cnf, :]
        y_test = YY.iloc[4000*(cnf-1):4000*cnf]
        
        X_train = XXX.drop(XXX.index[4000*(cnf-1):4000*cnf])
        y_train = YY.drop(YY.index[4000*(cnf-1):4000*cnf])
        
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_train)
        X_te = scaler.transform(X_test)
        
        pear_temp = []
        for m in range(1, 16):
            X_tr_sel = X_tr[:, :m]
            X_te_sel = X_te[:, :m]
            
            ridge = Ridge(alpha=0)
            ridge.fit(X_tr_sel, y_train)
            y_pr = ridge.predict(X_te_sel)
            pearson_corr_p, _ = pearsonr(y_pr, y_test)
            pear_temp.append(abs(pearson_corr_p))
        
        pear.append(pear_temp)
    
    avg_pear = np.mean(pear, axis=0)
    pearson_coff.append(avg_pear)
    print(f"Configuration {cnf} completed")

print("Experiment completed")

In [ ]:
Random_Results = np.array(pearson_coff)
np.save("Random_Results", Random_Results)

### Descripotrs vs propensity

In [11]:
Len = [0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
leng = len(Len)
Pearson_cnf = []

for cnf in range(1,11):
    Pearson = []
    for i in range(0,60):
        
            pearson_corr_p, p_value = pearsonr( XX.iloc[4000*(cnf-1):4000*cnf ,i],YY.iloc[4000*(cnf-1):4000*cnf])
            Pearson.append(pearson_corr_p)
    Pearson_cnf.append(Pearson)

In [12]:
Y3 = np.array(Pearson_cnf)
y3 = Y3.mean(axis=0)
np.save("descripotr_vs_propensity", Y3.mean(axis=0))